# Huewelerbach catchment

## Data preparation

We load the data from the `data` folder. We then cut them to the period of interest.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib qt
MARKERS_PROP = {'lw': 0, 'marker': 'o', 'color': 'black', 'markerfacecolor': 'none', 'markersize': 4}

In [ ]:
data = pd.read_csv('data/Huewelerbach.csv')
data.set_index(pd.to_datetime((data.year*10000+data.month*100+data.day).apply(str),format='%Y%m%d'), inplace=True)
data.drop(columns=['year', 'month', 'day'], inplace=True)
data.head()

In [ ]:
ax = data['Q(mm/d)'].plot(figsize=(10, 7))
ax.set_xlabel('Date')
ax.set_ylabel('Streamflow [mm/d]')
ax.set_ylim((0, ax.get_ylim()[1]))
ax.grid(True)

In [ ]:
START_SIMULATE = 0
START_VISUALIZE = 395 # Relative to START_SIMULATE
END_SIMULATE = 1124 # Absolute

P = data['P(mm/d)'].values[START_SIMULATE:END_SIMULATE]
E = data['E(mm/d)'].values[START_SIMULATE:END_SIMULATE]
Q_obs = data['Q(mm/d)'].values[START_SIMULATE:END_SIMULATE]
index = data.index[START_SIMULATE:END_SIMULATE]

## Model M01
### Initialization

We load the model and set inputs and time step. Calibrated parameters are contained in the dictionary `PARAMETERS_HUEWELERBACH_M01`

In [ ]:
from utils.m01 import m01, PARAMETERS_HUEWELERBACH_M01

m01.set_input([P, E])
m01.set_timestep(1.0)

### Explore the model

We show now some methods that can be used to visualize the model settings and to change them.

In [ ]:
print(m01)

In [ ]:
m01.get_parameters()

In [ ]:
m01.get_states()

In [ ]:
m01.set_parameters(PARAMETERS_HUEWELERBACH_M01)
PARAMETERS_HUEWELERBACH_M01

In [ ]:
m01.get_parameters()

In [ ]:
m01.set_states({'m01_FR_S0': 18.0})
m01.get_states()

In [ ]:
m01.reset_states()
m01.get_states()

In [ ]:
out = m01.get_output()
out

In [ ]:
m01.get_states()

In [ ]:
m01.get_internal(id='FR', attribute='state_array')

### Sensibility of the output to the parameters

We can use the function `parameters_sensitivity` to explore how the output varies changing the values of the parameters.

In [ ]:
from utils.sensitivity import parameters_sensitivity

In [ ]:
fig, ax = parameters_sensitivity(
    model=m01,
    default_parameters=PARAMETERS_HUEWELERBACH_M01, 
    par_name='m01_FR_k', 
    par_values=[1e-12, 1e-10, 1e-8, 1e-6]
)

ax.plot(Q_obs, **MARKERS_PROP)

In [ ]:
fig, ax = parameters_sensitivity(
    model=m01,
    default_parameters=PARAMETERS_HUEWELERBACH_M01, 
    par_name='m01_FR_alpha', 
    par_values=[1.0, 2.0, 3.0, 4.0, 5.0]
)

ax.plot(Q_obs, **MARKERS_PROP)

In [ ]:
fig, ax = parameters_sensitivity(
    model=m01,
    default_parameters=PARAMETERS_HUEWELERBACH_M01, 
    par_name='m01_FR_Ce', 
    par_values=[0.5, 0.75, 1.0, 1.25, 1.5]
)

ax.plot(Q_obs, **MARKERS_PROP)

### Plot calibrated results and states

We can observe the model behavior with the calibrated parameters. It is possible to call the `set_parameters` method to see how the output changes with different parameters.

In [ ]:
m01.reset_states()
m01.set_parameters(PARAMETERS_HUEWELERBACH_M01)
output = m01.get_output()[0]
state = m01.get_internal('FR', 'state_array').squeeze()
aet = m01.call_internal('FR', 'get_AET')[0]

In [ ]:
fig, ax = plt.subplots(3, 1, sharex=True)
ax[0].bar(range(len(P)), P)
ax[1].plot(output, label='Model output')
ax[1].plot(Q_obs, label='Observed', **MARKERS_PROP)
ax_bis = ax[1].twinx()
ax_bis.plot(state, label='Reservoir state', color='orange')
ax[2].plot(aet, label='AET')
ax[2].plot(E, label='PET', ls='--')
for a in ax:
    a.set_xlim((START_SIMULATE, len(P)))
    a.legend()
    
ax[0].set_ylabel('Precipitation [mm/d]')
ax[1].set_ylabel('Streamflow [mm/d]')
ax_bis.set_ylabel('State [mm]')
ax[2].set_ylabel('Evapotranspiration [mm/d]')

## Model M02
### Initialization

In [ ]:
from utils.m02 import m02, PARAMETERS_HUEWELERBACH_M02

m02.set_input([P, E])
m02.set_timestep(1.0)

### Explore the model

In [ ]:
print(m02)

### Sensibility of the output to the parameters

In [ ]:
PARAMETERS_HUEWELERBACH_M02

In [ ]:
fig, ax = parameters_sensitivity(
    model=m02,
    default_parameters=PARAMETERS_HUEWELERBACH_M02, 
    par_name='m02_FR_k',
    par_values=[1e-5, 1e-3, 1e-1]
)

ax.plot(Q_obs, **MARKERS_PROP)

In [ ]:
fig, ax = parameters_sensitivity(
    model=m02,
    default_parameters=PARAMETERS_HUEWELERBACH_M02, 
    par_name='m02_UR_Smax', 
    par_values=[10.0, 100.0, 1000.0]
)

ax.plot(Q_obs, **MARKERS_PROP)

### Plot calibrated results and states

In [ ]:
m02.reset_states()
m02.set_parameters(PARAMETERS_HUEWELERBACH_M02)
# m02.set_parameters({'m02_UR_Smax': 1000.0})
m02.set_states({'m02_UR_S0': 0.2*m02.get_parameters(['m02_UR_Smax'])['m02_UR_Smax']})
output_FR = m02.get_output()[0]
state_FR = m02.get_internal('FR', 'state_array').squeeze()
output_UR = m02.call_internal('UR', 'get_output', solve=False)[0]
state_UR = m02.get_internal('UR', 'state_array').squeeze()
aet = m02.call_internal('UR', 'get_AET')[0]

In [ ]:
fig, ax = plt.subplots(4, 1, sharex=True)
ax[0].bar(range(len(P)), P)
ax[1].plot(output_UR, label='Output UR')
ax_bis_1 = ax[1].twinx()
ax_bis_1.plot(state_UR, label='State UR', color='orange')
ax[2].plot(output_FR, label='Output FR')
ax[2].plot(Q_obs, label='Observed', **MARKERS_PROP)
ax_bis_2 = ax[2].twinx()
ax_bis_2.plot(state_FR, label='State FR', color='orange')
ax[3].plot(aet, label='AET')
ax[3].plot(E, label='PET', ls='--')
for a in ax:
    a.set_xlim((START_SIMULATE, len(P)))
    a.legend()
    
ax[0].set_ylabel('Precipitation [mm/d]')
ax[1].set_ylabel('Streamflow [mm/d]')
ax_bis_1.set_ylabel('State [mm]')
ax[2].set_ylabel('Streamflow [mm/d]')
ax_bis_2.set_ylabel('State [mm]')
ax[3].set_ylabel('Evapotranspiration [mm/d]')

## Model M03
### Initialization

In [ ]:
from utils.m03 import m03, PARAMETERS_HUEWELERBACH_M03

m03.set_input([P, E])
m03.set_timestep(1.0)

### Explore the model

In [ ]:
print(m03)

### Sensibility of the output to the parameters

In [ ]:
PARAMETERS_HUEWELERBACH_M03

In [ ]:
fig, ax = parameters_sensitivity(
    model=m03,
    default_parameters=PARAMETERS_HUEWELERBACH_M03, 
    par_name='m03_lag_lag-time', 
    par_values=[1.0, 5.0, 10.0]
)

ax.plot(Q_obs, **MARKERS_PROP)

### Plot calibrated results and states

In [ ]:
m03.reset_states()
m03.set_parameters(PARAMETERS_HUEWELERBACH_M03)
# m03.set_parameters({'m03_lag_lag-time': 5.0})
m03.set_states({'m03_UR_S0': 0.2*m03.get_parameters(['m03_UR_Smax'])['m03_UR_Smax']})
output_FR = m03.get_output()[0]
state_FR = m03.get_internal('FR', 'state_array').squeeze()
output_UR = m03.call_internal('UR', 'get_output', solve=False)[0]
state_UR = m03.get_internal('UR', 'state_array').squeeze()
output_lag = m03.call_internal('lag', 'get_output', solve=False)[0]
aet = m03.call_internal('UR', 'get_AET')[0]

In [ ]:
fig, ax = plt.subplots(5, 1, sharex=True)
ax[0].bar(range(len(P)), P)
ax[1].plot(output_UR, label='Output UR')
ax_bis_1 = ax[1].twinx()
ax_bis_1.plot(state_UR, label='State UR', color='orange')
ax[2].plot(output_UR, label='Input lag')
ax[2].plot(output_lag, label='Output lag')
ax[3].plot(output_FR, label='Output FR')
ax[3].plot(Q_obs, label='Observed', **MARKERS_PROP)
ax_bis_2 = ax[3].twinx()
ax_bis_2.plot(state_FR, label='State FR', color='orange')
ax[4].plot(aet, label='AET')
ax[4].plot(E, label='PET', ls='--')
for a in ax:
    a.set_xlim((START_SIMULATE, len(P)))
    a.legend()
    
ax[0].set_ylabel('Precipitation [mm/d]')
ax[1].set_ylabel('Streamflow [mm/d]')
ax_bis_1.set_ylabel('State [mm]')
ax[3].set_ylabel('Streamflow [mm/d]')
ax_bis_2.set_ylabel('State [mm]')
ax[4].set_ylabel('Evapotranspiration [mm/d]')

## Model M04
### Initialization

In [ ]:
from utils.m04 import m04, PARAMETERS_HUEWELERBACH_M04

m04.set_input([P, E])
m04.set_timestep(1.0)

### Explore the model

In [ ]:
print(m04)

### Sensibility of the output to the parameters

In [ ]:
PARAMETERS_HUEWELERBACH_M04

In [ ]:
fig, ax = parameters_sensitivity(
    model=m04,
    default_parameters=PARAMETERS_HUEWELERBACH_M04, 
    par_name='m04_split_split-par',
    par_values=[0, 0.12, 0.5, 1.0]
)

ax.plot(Q_obs, **MARKERS_PROP)

### Plot calibrated results and states

In [ ]:
m04.reset_states()
m04.set_parameters(PARAMETERS_HUEWELERBACH_M04)
m04.set_states({'m04_UR_S0': 0.2*m04.get_parameters(['m04_UR_Smax'])['m04_UR_Smax']})
output_total = m04.get_output()[0]
output_FR = m04.call_internal('FR', 'get_output', solve=False)[0]
state_FR = m04.get_internal('FR', 'state_array').squeeze()
output_UR = m04.call_internal('UR', 'get_output', solve=False)[0]
state_UR = m04.get_internal('UR', 'state_array').squeeze()
output_SR = m04.call_internal('SR', 'get_output', solve=False)[0]
state_SR = m04.get_internal('SR', 'state_array').squeeze()
aet = m04.call_internal('UR', 'get_AET')[0]

In [ ]:
fig, ax = plt.subplots(6, 1, sharex=True)
ax[0].bar(range(len(P)), P)
ax[1].plot(output_UR, label='Output UR')
ax_bis_1 = ax[1].twinx()
ax_bis_1.plot(state_UR, label='State UR', color='orange')
ax[2].plot(output_SR, label='Output SR')
ax_bis_2 = ax[2].twinx()
ax_bis_2.plot(state_SR, label='State SR', color='orange')
ax[3].plot(output_FR, label='Output FR')
ax_bis_3 = ax[3].twinx()
ax_bis_3.plot(state_FR, label='State FR', color='orange')
ax[4].plot(output_total, label='Output')
ax[4].plot(Q_obs, label='Observed', **MARKERS_PROP)
ax[5].plot(aet, label='AET')
ax[5].plot(E, label='PET', ls='--')

for a in ax:
    a.set_xlim((START_SIMULATE, len(P)))
    a.legend()
    
ax[0].set_ylabel('Precipitation [mm/d]')
ax[1].set_ylabel('Streamflow [mm/d]')
ax_bis_1.set_ylabel('State [mm]')
ax[2].set_ylabel('Streamflow [mm/d]')
ax_bis_2.set_ylabel('State [mm]')
ax[3].set_ylabel('Streamflow [mm/d]')
ax_bis_3.set_ylabel('State [mm]')
ax[4].set_ylabel('Streamflow [mm/d]')
ax[5].set_ylabel('Evapotranspiration [mm/d]')

## Compare different models

In [ ]:
out = []

for i, (m, par) in enumerate(zip([m01, m02, m03, m04], [PARAMETERS_HUEWELERBACH_M01, PARAMETERS_HUEWELERBACH_M02, PARAMETERS_HUEWELERBACH_M03, PARAMETERS_HUEWELERBACH_M04])):
    m.set_parameters(par)
    m.reset_states()
    if i == 1:
        m.set_states({'m02_UR_S0': 0.2*m.get_parameters(['m02_UR_Smax'])['m02_UR_Smax']})
    elif i == 2:
        m.set_states({'m03_UR_S0': 0.2*m.get_parameters(['m03_UR_Smax'])['m03_UR_Smax']})
    elif i == 3:
        m.set_states({'m04_UR_S0': 0.2*m.get_parameters(['m04_UR_Smax'])['m04_UR_Smax']})
    out.append(m.get_output()[0])

In [ ]:
fig, ax = plt.subplots(1, 1)

for i, o in enumerate(out):
    ax.plot(o, label='Model m0{}'.format(i+1))
    
ax.plot(Q_obs, **MARKERS_PROP, label='Observed')
ax.set_ylabel('Streamflow [mm/d]')
ax.set_xlim((START_SIMULATE, len(Q_obs)))
ax.grid(True)
ax.legend()

### Evaluate performance

We use Nash-Sutcliffe efficiency. It ranges between $-\infty$ and 1: 1 means perfect fit, values above 0.70 are considered good.

$$NSE(Q_{\rm{obs}}, Q_{\rm{sim}})=1-\frac{\sum \left(Q_{\rm{obs}}-Q_{\rm{sim}}\right)^2}{\sum \left(Q_{\rm{obs}}-\overline{Q_{\rm{obs}}}\right)^2}$$

In [ ]:
from utils.metrics import nse

for i, o in enumerate(out):
    nse_value = nse(obs=Q_obs[START_VISUALIZE:], sim=o[START_VISUALIZE:])
    print('NSE of m0{} is {:.3f}'.format(i+1, nse_value))